This is a notebook for data processing. This notebook loads the scraped data from CSV files, transforms and combines the data from different sources into one DataFrame object and saves the final dataset as a CSV file.

Project Repo: [SJR-Journal-Ranking](https://github.com/abir0/SJR-Journal-Ranking)

# Data Transformation

### Setup

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!gdown --id 1wxZYJh72U0SjMYhD2cnNBzcAmj3iifhr
!gdown --id 1hs9H-jcwwnUlxPKwM4X13mc02wVy0iNN
!gdown --id 1RJqeJiI5ks2WtEwuR9rKf10TS5ccbJvT

### SJR Data Transform

In [ ]:
sjr_df = pd.read_csv("/content/sjr_journal_ranking_2022.csv")

In [ ]:
sjr_df.columns

Remove duplicates and null values

In [ ]:
sjr_df.duplicated().sum()

In [ ]:
sjr_df.isnull().sum()

In [ ]:
sjr_df.dropna(inplace=True)

Remove unnessary columns

In [ ]:
sjr_df.drop(['URL'], axis=1, inplace=True)

Rename Columns

In [ ]:
sjr_df.rename(columns={'Rank': 'Best Subject Rank', 'Subject Area': 'Best Subject Area'}, inplace=True)

Change column data type

In [ ]:
sjr_df['Best Quartile'] = sjr_df['Best Quartile'].astype('category')

Ordered category for Quartiles

In [ ]:
sjr_df['Best Quartile'] = sjr_df['Best Quartile'].cat.set_categories(['Q1', 'Q2', 'Q3', 'Q4'], ordered=True)

Sort the data

In [ ]:
sjr_df.sort_values(['SJR index', 'Best Quartile', 'Best Subject Rank'], ascending=[False, True, True], inplace=True)

Remove `Title` duplicates

In [ ]:
sjr_df['Title'].duplicated().sum()

In [ ]:
sjr_df = sjr_df.drop_duplicates('Title')
sjr_df.reset_index(drop=True, inplace=True)

Add an `Overall Rank` based on the sorted index

In [ ]:
sjr_df["Overall Rank"] = sjr_df.index + 1

In [ ]:
sjr_df['Title Cap'] = sjr_df['Title'].apply(lambda x: x.upper())

### WoS Data Transform

In [ ]:
wos_df = pd.read_csv("/content/wos_master_journal_list.csv")
wos_df

Remove duplicates and null values

In [ ]:
wos_df.isnull().sum()

In [ ]:
wos_df.dropna(inplace=True)

In [ ]:
wos_df.duplicated().sum()

Merge SJR and WoS data based on the `Publisher` column

In [ ]:
wos_df.rename(columns={'Title': 'Title Cap'}, inplace=True)

In [ ]:
sjr_df = sjr_df.merge(wos_df, how='left', on='Title Cap')

Drop unnecessary columns

In [ ]:
sjr_df.drop(['Address'], axis=1, inplace=True)

Apply data transformations

In [ ]:
sjr_df['Publisher'] = sjr_df['Publisher'].apply(lambda x: str(x).title())

### Scopus Data Transform

In [ ]:
scopus_df = pd.read_excel("/content/scopus_citescore_2022.xlsx", sheet_name=0)

In [ ]:
scopus_df.columns

Rename columns

In [ ]:
scopus_df.rename(columns={'2022\nCiteScore': 'CiteScore',
       'Source Title (Medline-sourced journals are indicated in Green)': 'Title Cap',
       'Active or Inactive': 'Active Status',
       'Article language in source (three-letter ISO language codes)': 'ISO Language Codes',
       'Articles in Press included?': 'Articles in Press',
       'All Science Journal Classification Codes (ASJC)': 'ASJC Codes',
       'Top level:\n\nLife Sciences': 'Top level:Life Sciences',
       'Top level:\n\nSocial Sciences': 'Top level:Social Sciences',
       'Top level:\n\nPhysical Sciences': 'Top level:Physical Sciences',
       'Top level:\n\nHealth Sciences': 'Top level:Health Sciences',
       '1000 \nGeneral': 'General',
       '1100\nAgricultural and Biological Sciences': 'Agricultural and Biological Sciences',
       '1200\nArts and Humanities': 'Arts and Humanities',
       '1300\nBiochemistry, Genetics and Molecular Biology': 'Biochemistry, Genetics and Molecular Biology',
       '1400\nBusiness, Management and Accounting': 'Business, Management and Accounting',
       '1500\nChemical Engineering': 'Chemical Engineering',
       '1600\nChemistry': 'Chemistry',
       '1700\nComputer Science': 'Computer Science',
       '1800\nDecision Sciences': 'Decision Sciences',
       '1900\nEarth and Planetary Sciences': 'Earth and Planetary Sciences',
       '2000\nEconomics, Econometrics and Finance': 'Economics, Econometrics and Finance',
       '2100\nEnergy': 'Energy',
       '2200\nEngineering': 'Engineering',
       '2300\nEnvironmental Science': 'Environmental Science',
       '2400\nImmunology and Microbiology': 'Immunology and Microbiology',
       '2500\nMaterials Science': 'Materials Science',
       '2600\nMathematics': 'Mathematics',
       '2700\nMedicine': 'Medicine',
       '2800\nNeuroscience': 'Neuroscience',
       '2900\nNursing': 'Nursing',
       '3000\nPharmacology, Toxicology and Pharmaceutics': 'Pharmacology, Toxicology and Pharmaceutics',
       '3100\nPhysics and Astronomy': 'Physics and Astronomy',
       '3200\nPsychology': 'Psychology',
       '3300\nSocial Sciences': 'Social Sciences',
       '3400\nVeterinary': 'Veterinary',
       '3500\nDentistry': 'Dentistry',
       '3600\nHealth Professions': 'Health Professions',}, inplace=True)

Remove duplicates and null values and unecessary columns

In [ ]:
scopus_df.duplicated().sum()

In [ ]:
scopus_df['Title Cap'].duplicated().sum()

In [ ]:
scopus_df = scopus_df.drop_duplicates(subset=['Title Cap'])

In [ ]:
scopus_df.isnull().sum()

In [ ]:
scopus_df = scopus_df[scopus_df['Titles discontinued by Scopus due to quality issues'].isnull()]

In [ ]:
scopus_df['Titles discontinued by Scopus due to quality issues'].notnull().sum()

In [ ]:
scopus_df.drop(['Sourcerecord ID',
       'Print-ISSN', 'E-ISSN',
       'Titles discontinued by Scopus due to quality issues',
       'Medline-sourced Title? (see more info under separate tab)',
       'Open Access status',
       'Added to list May 2023', 'Source Type', 'Title history indication',
       'Related title to title history indication', 'Other related title 1',
       'Other related title 2', 'Other related title 3', 'Publisher\'s Name',
       'Publisher imprints grouped to main Publisher'], axis=1, inplace=True)

Apply data transformations

In [ ]:
scopus_df['Coverage'] = scopus_df['Coverage'].apply(lambda x: str(x).split(',')[-1].split('-')[0].strip())

In [ ]:
scopus_df['ASJC Codes'] = scopus_df['ASJC Codes'].apply(lambda x: x.split(';'))

In [ ]:
scopus_df['Active Status'] = scopus_df['Active Status'].apply(lambda x: True if x == "Active" else False)

Merge the Scopus data with the rest

In [ ]:
scopus_df['Title Cap'] = scopus_df['Title Cap'].apply(lambda x: x.upper())

In [ ]:
sjr_df = sjr_df.merge(scopus_df, how='left', on='Title Cap')

In [ ]:
sjr_df.drop(['Title Cap'], axis=1, inplace=True)

### Final Data

In [ ]:
sjr_df.info()

In [ ]:
sjr_df.head(6)

Save the data

In [ ]:
sjr_df.to_csv("combined_journal_ranking_data.csv")